In [3]:
import pandas as pd
import numpy as np
from scipy.stats import chisquare

In [4]:
draftdf = pd.read_csv('draft_data.csv')

In [5]:
performancedf = pd.read_csv('performance_data.csv')

In [6]:
draftdf.head()

,Unnamed: 0,Round,Pick,Overall,Team,Player,Position,Year,overall_pick,pick_points
0,0,1.0,1.0,2018-1,BUF,Rasmus Dahlin,D,2018,1,216
1,1,1.0,2.0,2018-2,CAR,Andrei Svechnikov,F,2018,2,215
2,2,1.0,3.0,2018-3,MTL,Jesperi Kotkaniemi,F,2018,3,214
3,3,1.0,4.0,2018-4,OTT,Brady Tkachuk,F,2018,4,213
4,4,1.0,5.0,2018-5,ARI,Barrett Hayton,F,2018,5,212


In [7]:
performancedf.head()

,Unnamed: 0,Year,Team,GP,W,L,year_diff,wins_change
0,232,2007,ANA,82,48,20,NaN,NaN
1,267,2008,ANA,82,47,27,1.0,-0.020833
2,313,2009,ANA,82,42,33,1.0,-0.106383
3,345,2010,ANA,82,39,32,1.0,-0.071429
4,372,2011,ANA,82,47,30,1.0,0.205128


In [13]:
draftdf['round_mult'] = 8 - draftdf['Round']

In [19]:
draftdf['pick_points'] = draftdf['round_mult'] * (39 - draftdf['Pick'])

In [22]:
performancedf['change_direction'] = 'positive'

In [24]:
mask = performancedf.wins_change < 0
performancedf.loc[mask, 'change_direction'] = 'negative'

In [25]:
performancedf.head()

,Unnamed: 0,Year,Team,GP,W,L,year_diff,wins_change,change_direction
0,232,2007,ANA,82,48,20,NaN,NaN,positive
1,267,2008,ANA,82,47,27,1.0,-0.020833,negative
2,313,2009,ANA,82,42,33,1.0,-0.106383,negative
3,345,2010,ANA,82,39,32,1.0,-0.071429,negative
4,372,2011,ANA,82,47,30,1.0,0.205128,positive


In [33]:
draftgroup1 = pd.DataFrame({'pick_points':draftdf.groupby(['Year', 'Team', 'Position'])['pick_points'].sum()}).reset_index()
draftgroup1

,Year,Team,Position,pick_points
0,1970,BOS,D,496.0
1,1970,BOS,F,1173.0
2,1970,BOS,G,312.0
3,1970,BUF,D,532.0
4,1970,BUF,F,1254.0
5,1970,BUF,G,76.0
6,1970,CHI,D,250.0
7,1970,CHI,F,675.0
8,1970,CHI,G,150.0
9,1970,CSE,D,340.0


In [34]:
draftgroup1['top_drafted'] = draftgroup1['Position']

In [40]:
draftgroup2 = pd.DataFrame({'pick_points':draftgroup1.groupby(['Year', 'Team'])['pick_points'].max()}).reset_index()
draftgroup2.head()

,Year,Team,pick_points
0,1970,BOS,1173.0
1,1970,BUF,1254.0
2,1970,CHI,675.0
3,1970,CSE,1087.0
4,1970,DET,972.0


In [43]:
draftgroup1.drop('top_drafted', axis=1, inplace=True)

In [44]:
draftgroup1.head()

,Year,Team,Position,pick_points
0,1970,BOS,D,496.0
1,1970,BOS,F,1173.0
2,1970,BOS,G,312.0
3,1970,BUF,D,532.0
4,1970,BUF,F,1254.0


In [51]:
topdrafted = draftgroup2.merge(draftgroup1, how='inner', on=['Team', 'Year', 'pick_points'])
topdrafted.head()

,Year,Team,pick_points,Position
0,1970,BOS,1173.0,F
1,1970,BUF,1254.0,F
2,1970,CHI,675.0,F
3,1970,CSE,1087.0,F
4,1970,DET,972.0,F


In [53]:
perform = performancedf[performancedf['year_diff'] == 1]
perform.head()

,Unnamed: 0,Year,Team,GP,W,L,year_diff,wins_change,change_direction
1,267,2008,ANA,82,47,27,1.0,-0.020833,negative
2,313,2009,ANA,82,42,33,1.0,-0.106383,negative
3,345,2010,ANA,82,39,32,1.0,-0.071429,negative
4,372,2011,ANA,82,47,30,1.0,0.205128,positive
5,419,2012,ANA,82,34,36,1.0,-0.276596,negative


In [55]:
perform_change = perform[['Year', 'Team', 'change_direction']].reset_index()
perform_change.head()

,index,Year,Team,change_direction
0,1,2008,ANA,negative
1,2,2009,ANA,negative
2,3,2010,ANA,negative
3,4,2011,ANA,positive
4,5,2012,ANA,negative


In [56]:
perform_change.drop('index', axis=1, inplace=True)

In [58]:
draft_direction = perform_change.merge(topdrafted, on=['Year', 'Team'], how='inner')
draft_direction.drop('pick_points', axis = 1, inplace=True)
draft_direction.head()

,Year,Team,change_direction,Position
0,2008,ANA,negative,F
1,2009,ANA,negative,D
2,2010,ANA,negative,F
3,2011,ANA,positive,F
4,2012,ANA,negative,D


In [66]:
final = draft_direction.groupby(['Position', 'change_direction']).count()
final.drop('C/LW', axis=0, inplace=True)
final.head()

Year  Team
Position change_direction            
D        negative           103   103
         positive           134   134
F        negative           372   372
         positive           370   370
G        negative             8     8

In [70]:
final['Year'].unstack()

change_direction,negative,positive
Position,,
D,103,134
F,372,370
G,8,6
